In [4]:
import os
import random
import string
import math
from PIL import Image, ImageDraw, ImageFont

images_directory = "./solid_dataset_od/images"
annotations_directory = "./solid_dataset_od/annotations"
os.makedirs(images_directory, exist_ok=True)
os.makedirs(annotations_directory, exist_ok=True)

# Specify font and font size
font_path = "/System/Library/Fonts/Supplemental/Times New Roman.ttf"
font_size = 100
font = ImageFont.truetype(font_path, font_size)

# Define image dimensions
image_width  = 200
image_height = 200

# Define the list of letters
letters = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"

In [5]:
# Function to create variations of an image
def create_variations(base_image, variation_count, variation_fn, xmin, ymin, xmax, ymax):
    variations = []
    for _ in range(variation_count):
        new_image = base_image.copy()
        new_image, new_xmin, new_ymin, new_xmax, new_ymax = variation_fn(new_image, xmin, ymin, xmax, ymax)
        location = new_xmin, new_ymin, new_xmax, new_ymax
        variations.append((new_image, location))
    return variations

# Function to resize the letter
def resize_letter(image, orig_xmin, orig_ymin, orig_xmax, orig_ymax):
    scale = random.uniform(0.8, 1.2)  # Scale between 0.8 and 1.2
    new_size = tuple(int(dim * scale) for dim in image.size)
    new_image = image.resize(new_size, resample=Image.BICUBIC)
    
    # Calculate new bounding box coordinates
    new_xmin = int(orig_xmin * scale)
    new_ymin = int(orig_ymin * scale)
    new_xmax = int(orig_xmax * scale)
    new_ymax = int(orig_ymax * scale)
    
    return new_image, new_xmin, new_ymin, new_xmax, new_ymax

# Function to move the letter within the image
def move_letter(image, orig_xmin, orig_ymin, orig_xmax, orig_ymax):
    x_offset = random.randint(-20, 20)
    y_offset = random.randint(-20, 20)
    new_image = Image.new("RGB", image.size, "black")
    new_image.paste(image, (x_offset, y_offset))
    
    # Calculate new bounding box coordinates
    new_xmin = orig_xmin + x_offset
    new_ymin = orig_ymin + y_offset
    new_xmax = orig_xmax + x_offset
    new_ymax = orig_ymax + y_offset
    
    return new_image, new_xmin, new_ymin, new_xmax, new_ymax

In [6]:
# Generate variations for each letter
for j in range(len(letters)):
    letter = letters[j]
    letter_images = []

    # Create base image with the hollow letter
    base_image = Image.new("RGB", (image_width, image_height), "black")
    
    draw = ImageDraw.Draw(base_image)
    text_width, text_height = draw.textsize(letter, font=font)
    x = (image_width  - text_width) // 2
    y = (image_height - text_height) // 2
    draw.text((x, y), letter, font=font, fill="red")
    
    letter_images.append(base_image)
    xmin, xmax, ymin, ymax = x, y, x + text_width, y + text_height
    letter_images_loc = [(xmin, ymin, xmax, ymax)]

    # Generate variations: rotation, resizing, and movement
    resizing_variations = create_variations(base_image, 10, resize_letter, xmin, ymin, xmax, ymax)
    movement_variations = create_variations(base_image, 10, move_letter, xmin, ymin, xmax, ymax)
    for img in resizing_variations:
        letter_images.append(img[0])
        letter_images_loc.append(img[1])
    for img in movement_variations:
        letter_images.append(img[0])
        letter_images_loc.append(img[1])
    
    image_folderpath = os.path.join(images_directory, f"{letter}")
    os.makedirs(image_folderpath, exist_ok = True)
    
    # Create annotations
    annotation_filename = os.path.join(annotations_directory, f"{letter}.csv")
    with open(annotation_filename, "w") as f:        
        # Save all variations
        for i, image in enumerate(letter_images):
            location = letter_images_loc[i]
            filename = f"image_{i+1}.png"
            startX, startY, endX, endY = location
            label  = letter
            
            image.save(os.path.join(image_folderpath, f"{filename}"))
            
            f.write(f"{filename},{startX},{startY},{endX},{endY},{label}\n")


/var/folders/5j/937d92fj6mb3d4_kbhlhjrfr0000gn/T/ipykernel_69303/14176721.py:10: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  text_width, text_height = draw.textsize(letter, font=font)
/var/folders/5j/937d92fj6mb3d4_kbhlhjrfr0000gn/T/ipykernel_69303/14176721.py:10: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  text_width, text_height = draw.textsize(letter, font=font)
/var/folders/5j/937d92fj6mb3d4_kbhlhjrfr0000gn/T/ipykernel_69303/14176721.py:10: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  text_width, text_height = draw.textsize(letter, font=font)
/var/folders/5j/937d92fj6mb3d4_kbhlhjrfr0000gn/T/ipykernel_69303/14176721.py:10: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlengt